<a href="https://colab.research.google.com/github/yxbxn/python/blob/main/naver_news_comment_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import sys
import pprint
import pandas as pd

In [ ]:
url = "https://n.news.naver.com/article/001/0013146102?sid=100"

url = "https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=001&aid=0013146102"

# 파라미터 분석
# sid1 - 분야 : 정치 100 / 경제 101 / 사회 102...
# oid - 신문사명 : 연합뉴스 001 / 프레시안 002 ...
# aid - 게시글 번호

In [ ]:
comment_list = []
sympathy_list = [] # 공감수
antipathy_list = [] # 비공감수
time_list = [] # 시간

# 신문사 명 및 게시글 번호 수집

oid = url.split("oid=")[1].split("&")[0]
aid = url.split("aid=")[1]
page = 1    
header = {
    "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
    "referer":url,
    
} 
while True :
    c_url="https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news"+oid+"%2C"+aid+"&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page="+str(page)+"&refresh=false&sort=FAVORITE"
    r = requests.get(c_url,headers = header)
    cont = BeautifulSoup( r.content,"html.parser")    
    total_comm = str(cont).split('comment":')[1].split(",")[0] # 총 댓글 개수
   
    match = re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
    # 리스트에 댓글 넣어주기
    comment_list.append(match)

    sympathy_list.append(re.findall('"sympathyCount":([^\*]*),"antipathyCount"', str(cont)))
    antipathy_list.append(re.findall('"antipathyCount":([^\*]*),"hideReplyButton"', str(cont)))
    time_list.append(re.findall('"modTime":([^\*]*),"modTimeGmt"', str(cont)))

    # 1페이지에 댓글 20개씩 여러 페이지 가져오기
    if int(total_comm) <= ((page) * 20):
        break
    else: 
        page += 1

In [ ]:
# 다중 리스트 단일 리스트로 변환
def flatten(lst):
    flatList = []
    for i in lst:
        if type(i) == list:
            for e in i:
                flatList.append(e) # 문자열 양 끝에 중복된 "" 제거해야함
        else:
            flatList.append(i)
    return flatList

In [ ]:
comment_df = pd.DataFrame()
comment_df["time"] = flatten(time_list)
comment_df["sympathy"] = flatten(sympathy_list)
comment_df["antipathy"] = flatten(antipathy_list)
comment_df["comment"] = flatten(comment_list)

In [ ]:
comment_df

## 간단한 전처리 수행하기
- 특수문자 제거하기
- 한글 자모음만 존재하는 것 제거하기 ( ㅋㅋㅋ ㅜㅜ )

In [ ]:
def new_data_cleansing_text(input,col):
    df = input.copy()

    # 줄바꿈 기호 삭제
    df[col] = df[col].apply(lambda x : x.replace(r"\n"," "))

    # 한글 자음모음 삭제
    df[col] = df[col].apply(lambda x : re.sub('([ㄱ-ㅎㅏ-ㅣ]+)',"",x))
        
    # 특수 기호 공백으로 처리
    df[col] = df[col].apply(lambda x : re.sub('[^\w\s]', ' ', x))

    # 다중공백 -> 단일공백
    df[col] = df[col].apply(lambda x : re.sub(' +', ' ', x))
        
    #양쪽 공백삭제
    df[col] = df[col].apply(lambda x : x.strip())
        
    return df

In [ ]:
comment_df_clean = new_data_cleansing_text(comment_df,"comment")

In [ ]:
comment_df.head()

In [ ]:
comment_df_clean.head()